In [7]:
import os
import pandas as pd

In [2]:
from run_evaluation import *

/Users/thomasflassbeck/Library/Caches/pypoetry/virtualenvs/parsee-datasets-TwOenO5C-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
base_path = "../data/by_sub_dataset_with_answers/rag-text"
multimodal = False

all_files = [x for x in os.listdir(base_path) if x.endswith(".csv")]
relevant_keys = {"total_correct_percent", "total_correct_meta_found_percent", "total_correct_percent_ex_missing", "total_correct_meta_found_percent_ex_missing", "total"}
# collect all results
all_results = {}
total_by_model = {}
for file in all_files:
    full_path = os.path.join(base_path, file)
    stats = run_eval_by_file(full_path, None, multimodal, use_cached_only=True)

    for model_name, results in stats.items():
        results["total"] = results["total_correct_percent"] if results["total_correct_meta_found_percent"] is None else ((results["total_correct_percent"] + results["total_correct_meta_found_percent"])/2)
        
        if model_name not in all_results:
            all_results[model_name] = {}
            total_by_model[model_name] = 0
        for key in relevant_keys:
            final_key = file.replace(".csv", "")+"_"+key
            all_results[model_name][final_key] = results[key]
            if key == "total":
                total_by_model[model_name] += results[key]

arr = []
for model_name, values in all_results.items():
    values = {"model": model_name, "total": total_by_model[model_name] / len(all_files), **values}
    arr.append(values)

df = pd.DataFrame(arr)
    

                                   model     total  \
0                 claude-3-opus-20240229  0.892545   
1                               assigned  1.000000   
2                     gpt-4-1106-preview  0.789279   
3         meta/meta-llama-3-70b-instruct  0.698243   
4          meta/meta-llama-3-8b-instruct  0.240000   
5  mistralai/Mixtral-8x22B-Instruct-v0.1  0.602432   
6               databricks/dbrx-instruct  0.574392   

   rag_text_50_rev23_meta_with_answers_total_correct_meta_found_percent_ex_missing  \
0                                               1.00                                 
1                                               1.00                                 
2                                               0.96                                 
3                                               0.94                                 
4                                               0.52                                 
5                                               0.8

In [14]:
df

,model,total,rag_text_50_rev23_meta_with_answers_total_correct_meta_found_percent_ex_missing,rag_text_50_rev23_meta_with_answers_total_correct_percent,rag_text_50_rev23_meta_with_answers_total_correct_meta_found_percent,rag_text_50_rev23_meta_with_answers_total_correct_percent_ex_missing,rag_text_50_rev23_meta_with_answers_total,rag_text_50_rev23_thousands_no_hint_with_answers_total_correct_meta_found_percent_ex_missing,rag_text_50_rev23_thousands_no_hint_with_answers_total_correct_percent,rag_text_50_rev23_thousands_no_hint_with_answers_total_correct_meta_found_percent,...,rag_text_50_rev_meta_with_answers_total_correct_meta_found_percent_ex_missing,rag_text_50_rev_meta_with_answers_total_correct_percent,rag_text_50_rev_meta_with_answers_total_correct_meta_found_percent,rag_text_50_rev_meta_with_answers_total_correct_percent_ex_missing,rag_text_50_rev_meta_with_answers_total,rag_text_50_rev22_millions_no_hint_with_answers_total_correct_meta_found_percent_ex_missing,rag_text_50_rev22_millions_no_hint_with_answers_total_correct_percent,rag_text_50_rev22_millions_no_hint_with_answers_total_correct_meta_found_percent,rag_text_50_rev22_millions_no_hint_with_answers_total_correct_percent_ex_missing,rag_text_50_rev22_millions_no_hint_with_answers_total
0,claude-3-opus-20240229,0.892545,1.00,0.96,1.00,0.96,0.98,None,0.86,None,...,0.956835,0.891892,0.898649,0.949640,0.895270,None,0.82,None,0.82,0.82
1,assigned,1.000000,1.00,1.00,1.00,1.00,1.00,None,1.00,None,...,1.000000,1.000000,1.000000,1.000000,1.000000,None,1.00,None,1.00,1.00
2,gpt-4-1106-preview,0.789279,0.96,0.88,0.96,0.88,0.92,None,0.64,None,...,0.934579,0.675676,0.675676,0.934579,0.675676,None,0.72,None,0.72,0.72
3,meta/meta-llama-3-70b-instruct,0.698243,0.94,0.88,0.94,0.88,0.91,None,0.56,None,...,0.148438,0.790541,0.128378,0.914062,0.459459,None,0.80,None,0.80,0.80
4,meta/meta-llama-3-8b-instruct,0.240000,0.52,0.32,0.52,0.32,0.42,None,0.22,None,...,NaN,NaN,NaN,NaN,NaN,None,0.12,None,0.12,0.12
5,mistralai/Mixtral-8x22B-Instruct-v0.1,0.602432,0.86,0.84,0.86,0.84,0.85,None,0.72,None,...,0.763158,0.297297,0.391892,0.578947,0.344595,None,0.38,None,0.38,0.38
6,databricks/dbrx-instruct,0.574392,0.64,0.66,0.64,0.66,0.65,None,0.50,None,...,0.589286,0.506757,0.445946,0.669643,0.476351,None,0.50,None,0.50,0.50
